In [1]:
from glob import glob
import json

In [2]:
files = glob('texts_1.jsonl*.splitted.requested')
files

['texts_1.jsonl03.splitted.requested',
 'texts_1.jsonl05.splitted.requested',
 'texts_1.jsonl02.splitted.requested',
 'texts_1.jsonl04.splitted.requested',
 'texts_1.jsonl07.splitted.requested',
 'texts_1.jsonl09.splitted.requested',
 'texts_1.jsonl14.splitted.requested',
 'texts_1.jsonl06.splitted.requested',
 'texts_1.jsonl08.splitted.requested',
 'texts_1.jsonl11.splitted.requested',
 'texts_1.jsonl00.splitted.requested',
 'texts_1.jsonl01.splitted.requested',
 'texts_1.jsonl10.splitted.requested',
 'texts_1.jsonl13.splitted.requested',
 'texts_1.jsonl12.splitted.requested']

In [3]:
mapping = {}
for f in files:
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            if 'Source text\nclear\nLook up details' in l['r']['result']:
                continue
            if 'clear\nLook up details' in l['r']['result']:
                continue
            if l['r']['result'].startswith('Source text\n'):
                continue
            mapping[l['src']] = l['r']['result']
            
len(mapping)

1498359

In [4]:
all_data = []
with open('train_1.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        data = []
        for d_ in l['data']:
            data.append({
                'value': d_,
                'value_ms': mapping.get(d_)
            })
        all_data.append(data)

In [5]:
all_data[0]

[{'value': 'How do different translations of biblical epistles impact their reception and meaning, and what are some notable differences between various translations?',
  'value_ms': 'Bagaimanakah terjemahan surat alkitabiah yang berbeza mempengaruhi penerimaan dan maknanya, dan apakah beberapa perbezaan ketara antara pelbagai terjemahan?'},
 {'value': 'Different translations of biblical epistles can have a significant impact on their reception and meaning because the choice of words, idioms, and grammar can influence the understanding of the message conveyed by the original text. Translators have to cope with linguistic, cultural, and theological challenges that require them to make judgments and decisions that might affect the accuracy, clarity, and readability of the text. Thus, the quality of a translation depends on the translator\'s expertise, methodology, and purpose, as well as the intended audience\'s needs and expectations.\n\nSome notable differences between various translat

In [6]:
with open('train_1.jsonl.translated', 'w') as fopen:
    for d in all_data:
        fopen.write(f'{json.dumps(d)}\n')

In [7]:
inside = []
with open('train_1.jsonl.translated') as fopen:
    for l in fopen:
        if 'model bahasa' in l or 'openai' in l:
            inside.append(l)
            if len(inside) >= 10:
                break

In [8]:
break_at = [
    'help.openai.com',
    'openai'
]

In [9]:
from tqdm import tqdm

all_texts = []
for i in tqdm(range(len(all_data))):
    texts = []
    for no, d in enumerate(all_data[i]):
        if d['value_ms'] is None:
            break
        if no % 2 == 0:
            t = '<manusia>: '
        else:
            t = '<bot>: '
        
        if any([b in d['value_ms'].lower() for b in break_at]):
            print(c['value_ms'], '\n-----\n')
            break
        
        n = d['value_ms']
        n = n.strip()
        if len(n) < 3:
            break
        t = t + n
        texts.append(t)

    if len(texts) % 2 != 0:
        texts = texts[:-1]
    if not len(texts):
        continue
    while len(texts) and texts[-1].startswith('<manusia>'):
        texts = texts[:-1]
    if not len(texts):
        continue
    if not texts[0].startswith('<manusia>'):
        continue
    all_texts.append('\n'.join(texts))

100%|████████████████████████████████| 150000/150000 [00:03<00:00, 47559.11it/s]


In [10]:
len(all_texts)

149992

In [11]:
all_texts[0]

'<manusia>: Bagaimanakah terjemahan surat alkitabiah yang berbeza mempengaruhi penerimaan dan maknanya, dan apakah beberapa perbezaan ketara antara pelbagai terjemahan?\n<bot>: Terjemahan surat-surat alkitabiah yang berbeza boleh memberi kesan yang ketara terhadap penerimaan dan maknanya kerana pilihan perkataan, simpulan bahasa, dan tatabahasa boleh mempengaruhi pemahaman mesej yang disampaikan oleh teks asal. Penterjemah perlu menghadapi cabaran linguistik, budaya dan teologi yang memerlukan mereka membuat pertimbangan dan keputusan yang mungkin menjejaskan ketepatan, kejelasan dan kebolehbacaan teks. Oleh itu, kualiti terjemahan bergantung kepada kepakaran, metodologi, dan tujuan penterjemah, serta keperluan dan jangkaan khalayak yang dimaksudkan.\n\nBeberapa perbezaan ketara antara pelbagai terjemahan surat alkitab termasuk penggunaan bahasa formal atau tidak formal, perbendaharaan kata kuno atau moden, kesetaraan literal atau dinamik, dan berat sebelah sektarian atau ekumenikal. C

In [12]:
for t in all_texts:
    splitted = t.split('<manusia>:')
    if '<bot>:' not in splitted[-1]:
        print(t)
        break

In [13]:
with open('part1_texts.jsonl', 'w') as fopen:
    for t in all_texts:
        fopen.write(f'{json.dumps(t)}\n')